In [ ]:
# for loading/processing the images  
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input 

# models 
from keras.applications.vgg16 import VGG16 
from keras.models import Model

# clustering and dimension reduction
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# for everything else
import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle
import tensorflow as tf
from PIL import Image

In [ ]:
def img_cls(folder):
    img = []
    cls = []
    filename = []
    for i in os.listdir(folder):
        
        x = 20
        for file in os.listdir(os.path.join(folder, i)):
            img_path = os.path.join(folder, i, file)
            filename.append(img_path)
            while x > 0:               
                
                #one_img = np.resize(np.array(Image.open(img_path)), (224, 224, 3)).astype('float32')/255
                image = np.array(Image.open(img_path).resize((224,224))).astype('float32')/255
#                 image = Image.open(img_path)
#                 image = np.array(image)
#                 image = image.resize((224, 224, 3))
#                 image = image.astype('float32')/255
                img.append(image)
                cls.append(i)
                x -= 1
    return img, cls, filename

In [ ]:
img_data, cls_data, filename = img_cls('C:/Users/gc/Documents/GA/Projects/Project_5/imag')

In [ ]:
img_data[:5]

In [ ]:
cls_data[:5]

In [ ]:
#filename

In [ ]:
reshaped_img = [image.reshape(1, 224, 224, 3) for image in img_data]

In [ ]:
reshaped_img[0].shape

In [ ]:
preprocessed_img = [preprocess_input(image) for image in reshaped_img]


In [ ]:
preprocessed_img[0][:5]

In [ ]:
# load pre-trained model
model = VGG16()
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)

In [ ]:

features = [model.predict(image, use_multiprocessing=True) for image in preprocessed_img]

In [ ]:
features = np.array(features)

In [ ]:
features_reshape = features.reshape(-1, 4096)

In [ ]:
features_reshape.shape

In [ ]:
# reduce the amount of dimensions in the feature vector
pca = PCA(n_components=20, random_state=22)
pca.fit(features_reshape)
x = pca.transform(features_reshape)

In [ ]:
x

In [ ]:
# cluster feature vectors
kmeans = KMeans(n_clusters=20,n_jobs=-1, random_state=22)
kmeans.fit(x)

In [ ]:
kmeans.labels_

In [ ]:
# holds the cluster id and the images { id: [images] }
groups = {}
for file, cluster in zip(filename,kmeans.labels_):
    if cluster not in groups.keys():
        groups[cluster] = []
        groups[cluster].append(file)
    else:
        groups[cluster].append(file)

# function that lets you view a cluster (based on identifier)        
def view_cluster(cluster):
    plt.figure(figsize = (25,25));
    # gets the list of filenames for a cluster
    files = groups[cluster]
    print(len(files))
    # only allow up to 30 images to be shown at a time
    if len(files) > 30:
        print(f"Clipping cluster size from {len(files)} to 30")
        files = files[:29]
    # plot each image in the cluster
    for index, file in enumerate(files):
        plt.subplot(10,10,index+1);
        img = load_img(file)
        img = np.array(img)
        plt.imshow(img)
        plt.axis('off')

In [ ]:
print(groups[0])

In [ ]:
view_cluster(1)